In [30]:
import pandas as pd
import os

In [39]:
in_path = "/data/clusterfs/lag/users/jitame/SENT_CORE/geno/regenie/exome/functional_annotation/selected_variants_v1"
run_name = "AICHA_rs_conn"
set_size=200
save_lists=False
run_all=True
#chr_no=3

def split_sets(in_path, file_name_ext, run_name, set_size, chr_no, save_lists=True):
    """
    Subdivides all genes for one chromosome into smaller sets in order to improve parallellization
    """
    
    #load data
    all_sets = pd.read_csv(os.path.join(in_path, file_name_ext), sep="\t", names="Gene CHR POS VARS".split())
    
    #get number of sets
    no_sets = ( len(all_sets) // set_size ) + 1

    if save_lists:
        for i in range( no_sets ):      
            #get set 
            this_set = list(all_sets.Gene[ (i*set_size):((i+1)*set_size)])
            print(this_set[0])

            #save set
            with open(os.path.join(in_path, "{0}_c{1}_extract_set_{2}.txt".format(run_name, chr_no, i, no_sets)), 'w') as file:
                for row in this_set:
                    s = "".join(map(str, row))
                    file.write(s+'\n')

    return chr_no, no_sets

df_all_sets = pd.DataFrame(columns="Chr_no No_sets".split())

if run_all:
    for c in range(1, 23, 1):
        df_all_sets.loc[len(df_all_sets)] = split_sets(in_path = in_path,
                                                       file_name_ext = "{0}_c{1}_set_list.txt".format(run_name, c) ,
                                                       run_name = run_name,
                                                       set_size = set_size,
                                                       chr_no = c,
                                                       save_lists=save_lists)
    c="X"
    df_all_sets.loc[len(df_all_sets)] = split_sets(in_path = in_path,
                                                       file_name_ext = "{0}_c{1}_set_list.txt".format(run_name, c) ,
                                                       run_name = run_name,
                                                       set_size = set_size,
                                                       chr_no = c,
                                                       save_lists=save_lists)


In [48]:
def make_command_file(df, file_name=None):
    command_list = []
    command_string = "bash ~/bin/code/AICHA/03_regenie/03_exome/exome_regenie_step2_gene_based_submitter_additive_sum_aicha.sh {0} {1}"
    for ind in df.index.values:
        for s_no in range(df.loc[ind,"No_sets"]):
            command_list.append(command_string.format(df.loc[ind,"Chr_no"], s_no))
        
    if file_name is not None:
        with open(file_name, 'w') as file:
            for row in command_list:
                s = "".join(map(str, row))
                file.write(s+'\n')

make_command_file(df_all_sets, file_name = "/data/workspaces/lag/workspaces/lg-ukbiobank/projects/rest-multimodal/wes/test_run_all_command.sh")

['bash ~/bin/code/AICHA/03_regenie/03_exome/exome_regenie_step2_gene_based_submitter_additive_sum_aicha.sh 1 0', 'bash ~/bin/code/AICHA/03_regenie/03_exome/exome_regenie_step2_gene_based_submitter_additive_sum_aicha.sh 1 1', 'bash ~/bin/code/AICHA/03_regenie/03_exome/exome_regenie_step2_gene_based_submitter_additive_sum_aicha.sh 1 2', 'bash ~/bin/code/AICHA/03_regenie/03_exome/exome_regenie_step2_gene_based_submitter_additive_sum_aicha.sh 1 3', 'bash ~/bin/code/AICHA/03_regenie/03_exome/exome_regenie_step2_gene_based_submitter_additive_sum_aicha.sh 1 4', 'bash ~/bin/code/AICHA/03_regenie/03_exome/exome_regenie_step2_gene_based_submitter_additive_sum_aicha.sh 1 5', 'bash ~/bin/code/AICHA/03_regenie/03_exome/exome_regenie_step2_gene_based_submitter_additive_sum_aicha.sh 1 6', 'bash ~/bin/code/AICHA/03_regenie/03_exome/exome_regenie_step2_gene_based_submitter_additive_sum_aicha.sh 1 7', 'bash ~/bin/code/AICHA/03_regenie/03_exome/exome_regenie_step2_gene_based_submitter_additive_sum_aicha.

In [28]:
df_all_sets

,Chr_no,No_sets
0,1,10
1,2,6
2,3,6
3,4,4
4,5,5
5,6,6
6,7,5
7,8,4
8,9,4
9,10,4


In [25]:
df_all_sets.No_sets.sum()

103